In [ ]:
# WorkFlowWith.ipynb

import easyocr

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

def perform_ocr(image_path):
    """Performs OCR on the given image and returns extracted text."""
    results = reader.readtext(image_path)
    extracted_text = " ".join([detection[1] for detection in results])
    return extracted_text


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [9]:
import subprocess
import json

def query_llama(prompt):
    """Sends a prompt to the LLaMA model and returns the response."""
    try:
        result = subprocess.run(
            ["ollama", "run", "llama3.1"], 
            input=prompt,
            text=True,
            encoding="utf-8",
            capture_output=True,
            check=True,
        )
        return result.stdout.strip()
    except subprocess.CalledProcessError as e:
        print("Error interacting with Ollama server:", e.stderr)
        return None

def perform_llm(extracted_text):
    """Prepares the prompt and retrieves structured output from the LLaMA model."""
    prompt = f"""
    You are an advanced medical assistant. Validate and transform the following unstructured medical text into a structured JSON format with these sections:
    1. Patient Information
    2. Date of Issue
    3. Type of Report (Heart, Brain, Skin, Bones, etc.)
    4. Medical Problem (technical terms for doctors)
    5. Simplified Explanation of the Medical Problem (for non-experts)

    Unstructured Medical Text:
    {extracted_text}

    Output only the following JSON:
    {{
        "Patient Information": "string",
        "Date of Issue": "string",
        "Type of Report": "string",
        "Medical Problem": "string",
        "Simplified Explanation": "string"
    }}
    """

    response = query_llama(prompt)
    return response


In [7]:
# Function to save the structured output as a JSON file
def save_json_to_file(data, filename="output.json"):
    with open(filename, "w", encoding="utf-8") as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)
        print(f"JSON saved to {filename}")

In [5]:
import re

def extract_json(response):
    """Extracts JSON part from the response."""
    # Regular expression to match valid JSON block
    json_match = re.search(r'\{.*\}', response, re.DOTALL)
    
    if json_match:
        try:
            # Parse the matched JSON string
            structured_output = json.loads(json_match.group(0))
            save_json_to_file(structured_output)
            return structured_output
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            return None
    else:
        print("No valid JSON found in the response.")
        return None